<a href="https://colab.research.google.com/github/KenyanGeek/Predicting-Loan-Default/blob/master/Predicting_Loan_Default_for_Zindi_Using_Fast_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing the Data

# Predicting Loan Default using Fast.ai

In [0]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [3]:
from google.colab import files
uploaded = files.upload()


Saving data_train.csv to data_train.csv


In [4]:
import io


df_train = pd.read_csv(io.BytesIO(uploaded['data_train.csv']))

df_train

,Unnamed: 0,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,systemloanid_x,loannumber_x,approveddate_x,creationdate_x,loanamount_x,totaldue_x,termdays_x,referredby_x,good_bad_flag,systemloanid_y,loannumber_y,approveddate_y,creationdate_y,loanamount_y,totaldue_y,termdays_y,closeddate,referredby_y,firstduedate,firstrepaiddate
0,0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN,301964962,2,29:48.0,29:42.0,10000,13000.0,30,8a858899538ddb8e0153a780c56e34bb,Good,301943951,1,55:26.0,54:19.0,10000,11500.0,15,09:47.0,8a858899538ddb8e0153a780c56e34bb,00:00.0,59:36.0
1,1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN,301972172,2,21:46.0,21:40.0,10000,13000.0,30,NaN,Bad,301929966,1,47:30.0,46:22.0,10000,13000.0,30,52:54.0,NaN,00:00.0,42:44.0
2,2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN,301976271,4,40:27.0,40:19.0,10000,13000.0,30,NaN,Good,301931228,3,49:34.0,49:27.0,10000,13000.0,30,12:20.0,NaN,00:00.0,02:11.0
3,3,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN,301976271,4,40:27.0,40:19.0,10000,13000.0,30,NaN,Good,301895253,1,07:01.0,06:40.0,10000,11500.0,15,02:53.0,NaN,00:00.0,52:45.0
4,4,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN,301976271,4,40:27.0,40:19.0,10000,13000.0,30,NaN,Good,301919784,2,10:41.0,10:34.0,10000,11500.0,15,13:50.0,NaN,00:00.0,03:40.0
5,5,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN,301997763,2,03:17.0,03:09.0,10000,11500.0,15,NaN,Good,301939781,1,05:26.0,04:18.0,10000,11500.0,15,24:13.0,NaN,00:00.0,14:05.0
6,6,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301876300,3,59:19.0,59:13.0,10000,13000.0,30,45:02.0,NaN,00:00.0,29:52.0
7,7,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301953421,5,07:32.0,06:24.0,20000,24500.0,30,57:17.0,NaN,00:00.0,47:08.0
8,8,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301916071,4,56:10.0,56:04.0,20000,24500.0,30,22:04.0,NaN,00:00.0,11:56.0
9,9,8a858ea05a859123015a8892914d15b7,1990-07-21 00:00:00.000000,Savings,3.365935,6.564823,Access Bank,NaN,Permanent,NaN,301992494,6,44:43.0,44:36.0,20000,24500.0,30,NaN,Good,301849134,2,10:12.0,10:04.0,10000,13000.0,30,45:58.0,NaN,00:00.0,30:48.0


In [0]:
dep_var = 'good_bad_flag'
cat_names = ['birthdate','customerid','longitude_gps','latitude_gps','bank_account_type','bank_name_clients','bank_branch_clients','employment_status_clients','level_of_education_clients','systemloanid_x','loannumber_x','termdays_x','referredby_x','loannumber_y','termdays_y','referredby_y','approveddate_x','creationdate_x','creationdate_y','closeddate','firstduedate','firstrepaiddate']
cont_names = ['loanamount_x','totaldue_x','loanamount_y','totaldue_y']
procs = [FillMissing, Categorify, Normalize]

In [6]:
from google.colab import files
uploaded = files.upload()


Saving data_test.csv to data_test.csv


In [0]:
import io


df_test = pd.read_csv(io.BytesIO(uploaded['data_test.csv']))

test = TabularList.from_df(df_test, cat_names=cat_names, cont_names=cont_names, procs=procs) 


In [0]:
data = (TabularList.from_df(df_train, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_rand_pct(0.2)
                           .label_from_df(cols=dep_var)
                           .add_test(test, label = 'good_bad_flag')
                           .databunch())


In [9]:
data.show_batch(rows=10)

birthdate,customerid,longitude_gps,latitude_gps,bank_account_type,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients,systemloanid_x,loannumber_x,termdays_x,referredby_x,loannumber_y,termdays_y,referredby_y,approveddate_x,creationdate_x,creationdate_y,closeddate,firstduedate,firstrepaiddate,loanamount_x,totaldue_x,loanamount_y,totaldue_y,target
1994-07-22 00:00:00.000000,8a858fe8555f401f01555f4a125f01ac,3.2292977,6.4726109,Savings,FCMB,#na#,Permanent,Post-Graduate,301997104,13,90,#na#,10,60,#na#,36:11.0,36:04.0,19:35.0,41:33.0,00:00.0,52:17.0,1.9174,2.0020,1.9572,2.1087,Good
1987-03-03 00:00:00.000000,8a858e5b5a80188b015a81895c596e21,3.21171,6.9180192,Savings,Stanbic IBTC,#na#,Permanent,#na#,301960870,4,30,#na#,2,15,#na#,27:24.0,27:18.0,51:11.0,30:04.0,00:00.0,19:56.0,-1.2455,-1.2233,-0.7034,-0.7785,Good
1975-04-17 00:00:00.000000,8a858e5f5a41b97f015a4212766e2f84,3.9100154,7.4377675,Savings,Access Bank,#na#,#na#,#na#,301965525,13,30,#na#,11,30,#na#,39:11.0,39:06.0,29:00.0,06:18.0,00:00.0,56:09.0,1.1267,1.0236,2.4893,2.3082,Good
1982-02-14 00:00:00.000000,8a858f385b34d285015b352600354046,6.319015,4.9327642,Savings,Zenith Bank,#na#,Permanent,#na#,301959319,4,30,#na#,1,30,#na#,15:43.0,15:37.0,56:32.0,57:37.0,00:00.0,42:29.0,-0.4548,-0.3898,-0.7034,-0.6361,Good
1976-05-20 00:00:00.000000,8a8588fd53c244bc0153c7b3845e0983,3.3736899,6.4450746,Other,Fidelity Bank,#na#,Permanent,Graduate,301994661,14,60,#na#,6,15,#na#,16:05.0,15:58.0,04:26.0,37:43.0,00:00.0,22:34.0,1.1267,1.3135,-0.1713,-0.2858,Bad
1990-02-16 00:00:00.000000,8a858f7a58c218eb0158c8e76932370e,3.1914976000000004,6.6626789,Savings,Diamond Bank,#na#,Permanent,#na#,302001752,8,30,#na#,5,30,#na#,01:52.0,00:20.0,26:30.0,13:48.0,00:00.0,03:39.0,-0.8501,-0.9153,0.3608,0.3422,Good
1974-10-26 00:00:00.000000,8a858e5856b7840c0156c1158b2c4a07,3.2758098,6.570848599999999,Other,GT Bank,#na#,Self-Employed,Graduate,301976727,10,30,#na#,1,30,#na#,08:47.0,07:37.0,44:20.0,28:09.0,00:00.0,13:00.0,0.3360,0.3350,-0.7034,-0.6361,Good
1988-10-22 00:00:00.000000,8a858ff35acdd593015ad12db5593535,3.3572828,6.586772099999999,Savings,GT Bank,#na#,Permanent,#na#,301960175,8,30,#na#,4,30,#na#,10:09.0,09:01.0,16:43.0,14:26.0,00:00.0,59:17.0,0.3360,0.3350,0.3608,0.3897,Good
1983-10-01 00:00:00.000000,8a858ff35acdd593015acf094819393e,7.4341300000000015,9.0718133,Savings,GT Bank,#na#,Permanent,#na#,301980080,5,30,#na#,1,30,#na#,25:33.0,25:25.0,46:23.0,17:08.0,00:00.0,01:57.0,-1.2455,-1.2233,-0.7034,-0.6361,Good
1976-02-28 00:00:00.000000,8a858ee755a0c84a0155a3b5a9ab2308,7.4100712,10.4378213,Savings,Zenith Bank,#na#,Permanent,Secondary,301968884,10,15,#na#,7,15,#na#,49:41.0,49:35.0,45:59.0,28:11.0,00:00.0,13:02.0,-1.2455,-1.3320,-0.7034,-0.7785,Bad


In [0]:
learn = tabular_learner(data, layers=[1000,500], metrics=accuracy)

In [11]:
learn.fit(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.131334,0.070231,0.975530,00:28


## Here i can predict with 97% accuracy

## Next is to predict on all the test data and save that to an excel

In [0]:
probs = learn.get_preds(ds_type=DatasetType.Test)[0]

def probs2class(item): 
    return max(range(len(item)), key=item.__getitem__) 

test_df = pd.DataFrame({'customerid': df_test['customerid'], 'target': list(map(probs2class, probs))})
test_df.to_csv('final.csv', header=True, index=False)

